In [3]:
SEASON = '2017-18'

In [4]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,James Harden,HOU,SG,19.609161,28,72,72,35.4,9.0,20.1,...,36.1,11.6,3.8,15.4,0.289,8.5,1.4,9.9,7.7,65.0
2,LeBron James,CLE,PF,17.801883,33,82,82,36.9,10.5,19.3,...,31.6,11.0,3.0,14.0,0.221,7.3,1.4,8.7,8.2,50.0
3,Anthony Davis,NOP,PF,14.846424,24,75,75,36.4,10.4,19.5,...,30.0,8.8,4.9,13.7,0.241,5.3,1.4,6.7,5.9,48.0
4,Damian Lillard,POR,PG,13.953656,27,73,73,36.6,8.5,19.4,...,30.6,9.9,2.7,12.6,0.227,7.0,0.3,7.2,6.3,49.0
5,Russell Westbrook,OKC,PG,13.935775,29,80,80,36.4,9.5,21.1,...,34.1,5.5,4.5,10.1,0.166,4.8,1.5,6.3,6.1,48.0
6,Kevin Durant,GSW,SF,13.681647,29,68,68,34.2,9.3,18.0,...,30.4,7.5,2.9,10.4,0.215,6.7,0.6,7.3,5.5,58.0
7,Giannis Antetokounmpo,MIL,PF,13.363241,23,75,75,36.7,9.9,18.7,...,31.2,8.3,3.6,11.9,0.207,5.3,0.9,6.2,5.7,44.0
8,Karl-Anthony Towns,MIN,C,12.501753,22,82,82,35.6,7.8,14.3,...,22.9,10.6,3.4,14.0,0.230,4.9,0.2,5.1,5.2,47.0
9,Stephen Curry,GSW,PG,12.039868,29,51,51,32.0,8.4,16.9,...,31.0,7.2,1.9,9.1,0.267,7.8,0.0,7.7,4.0,58.0
10,Nikola Jokić,DEN,C,11.974290,22,75,73,32.6,6.7,13.5,...,24.2,7.8,2.9,10.7,0.211,5.3,1.6,6.9,5.5,46.0
